<a href="https://colab.research.google.com/github/MLcmore2023/MLcmore2023/blob/main/day3_pm_afternoon/random-forest-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest
Random forest is an ensemble machine learning algorithm that combines multiple decision trees to make accurate classification and regression. It works by training numerous decision trees on different subsets of the dataset using bootstrapping (random sampling with replacement) and feature randomization. These trees collectively form a "forest," and their predictions are averaged. Random forests are also non-parametric and require little to no parameter tuning. They differ from many common machine learning models used today that are typically optimized using gradient descent.

<img src="https://www.tibco.com/sites/tibco/files/media_entity/2021-05/random-forest-diagram.svg">


### Import libraries and initialize random generator

In [254]:
import numpy as np
from sklearn.datasets import fetch_openml # for loadin dataset
# Set the seed value to make the random number reproducible
np.random.seed(0)

### Load data using `sklearn` library
The sonar dataset contains data collected from sonar signals that were used to discriminate between underwater objects as either "rocks" or "mines." The dataset consists of 208 observations, each represented by 60 features from sonar signals. The signals are obtained from a variety of different aspect angles. The goal is to classify these signals and distinguish between `rocks` and `mines`.

<img src="https://storage.googleapis.com/kaggle-datasets-images/1662635/2727659/3493b9309a1cf4f0c07aa6175b820060/dataset-card.jpg?t=2021-11-13-14-25-59" width=20%>

In [255]:
# Load the Sonar dataset from scikit-learn's datasets
sonar_data = fetch_openml(name='sonar', version=1, as_frame=True)

# The dataset is loaded as a dictionary-like object
X = sonar_data['data']    # Feature matrix
y = sonar_data['target']  # Target values

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [256]:
X

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_51,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


In [257]:
y

0      Rock
1      Rock
2      Rock
3      Rock
4      Rock
       ... 
203    Mine
204    Mine
205    Mine
206    Mine
207    Mine
Name: Class, Length: 208, dtype: category
Categories (2, object): ['Mine', 'Rock']

There are two classes in our dataset: Rock and Mine. The labels are in strings, so we will convert these categorical labels from strings (e.g., "Rock" and "Mine") into 0 and 1. By converting strings to numbers, the models can process the data more efficiently.

In [258]:
mapping = {"Rock": 0, "Mine": 1}

# Use the map function to apply the mapping and convert strings to integers
y = y.map(mapping)
y

0      0
1      0
2      0
3      0
4      0
      ..
203    1
204    1
205    1
206    1
207    1
Name: Class, Length: 208, dtype: category
Categories (2, int64): [1, 0]

we now convert the dataset into numpy arrays so later we can perform calculations on them

In [259]:
X = X.to_numpy()
y = y.to_numpy()

### Split the dataset into training sets and testing sets

In [260]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# REVIEW: decision trees
Random forest is made out of decision trees, so we will first review decision trees.
<img src="https://raw.githubusercontent.com/MLcmore2023/MLcmore2023/main/.images/decisiontree1.png" width=60%>


We will use `sklearn` library's decision tree to make predictions on the sonar dataset.

In [261]:
from sklearn.tree import DecisionTreeClassifier

# Create and train the Decision Tree model
decision_tree = DecisionTreeClassifier(max_depth=None)
decision_tree.fit(X_train, y_train)

# Make predictions on the test set
predictions = decision_tree.predict(X_test)

# See result
print(y_test)
print(predictions)

[0 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1
 1 0 1 1 0]
[1 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1
 1 1 1 1 0]


a big problem with decision tree is that they are not accuracy on complicated datasets. We can see that the prediction is very different than the actual y. The accuracy is very low.

In [262]:
print(y_test==predictions)

[False False False  True  True False  True  True False False  True  True
  True  True  True  True  True  True  True  True False False False False
  True  True False  True  True  True  True False  True  True  True  True
  True  True False  True  True  True]


In [263]:
N = len(y_test)
count_of_correct_predictions = sum( y_test==predictions  )
accuracy = count_of_correct_predictions/N
print(accuracy)

0.6904761904761905


### Initialize model
Decision trees are easy to make and easy to interpret. However, in the real world, they are usually NOT accuracy on complex dataset. Random forest solve this problem by have many decision trees, and combining the result.

Therefore, in random forest, the model is simply a list of individual decision trees.
` [🌳0, 🌳1, 🌳2, 🌳3... ]`

To initialize the model, we just make an empty list

In [264]:
random_forest_model = [] # empty list

### Bootstrapping
One of the main reasons random forests are a powerul machine learning model is the idea behind injecting randomness into each tree. Each individual decision tree will be constructed on a "bootstrapped" subset of our data. If our dataset has $n$ observations "bootstrapping" is the process of sampling $n$ points **with** replacement. The probability an observation is omitted from our bootstrapped dataset is  $(1 - \frac{1}{n})^{n}$. $e^{-1} = \displaystyle \lim_{n\to\infty}(1-\frac{1}{n})^n$ and since $e^{-1}$ = 0.36787.. $\approx \frac{1}{3}$ $\Rightarrow$ bootstrapping $n$ samples with replacement will leave out approximately $\frac{1}{3}$ of the observations in each distinct tree. Since each individual tree is built using only $\frac{2}{3}$ of the data, each tree will be different from each other.

In [265]:
example_list = np.array(['a','b','c','d','e','f','g','h'])
N = 8

In [266]:
bootstrap_indices = np.random.choice(N, N, replace=True)
bootstrap_indices.sort() # sort the array so it is easier for us to read (this is not necessary)
print(bootstrap_indices)

[0 0 2 3 3 4 5 7]


In [267]:
bootstrap_sample = example_list[bootstrap_indices]
print(bootstrap_sample)

['a' 'a' 'c' 'd' 'd' 'e' 'f' 'h']


In [268]:
def bootstrap_sampling(X, y):
    N = X.shape[0] # how many entries are there in the dataset
    bootstrap_indices = np.random.choice(N, N, replace=True)

    X_bootstrap = X[bootstrap_indices]
    y_bootstrap = y[bootstrap_indices]

    return X_bootstrap, y_bootstrap

### Making a random forest
There are 3 steps:
1. Decide how many trees we want. We will use 100 for here.
2. For every tree, first make a bootstrap sample from the data
3. Make the decision tree and append it to the model

In [269]:
n_estimators = 100 # The number of trees in the forest

def train_random_forest(X, y, random_forest_model):
    n_samples, n_features = X.shape

    for i in range(n_estimators):
        # Bootstrap sampling
        X_bootstrap, y_bootstrap = bootstrap_sampling(X,y)

        # Create a Decision Tree and fit it to the bootstrap sample
        tree = DecisionTreeClassifier(max_depth=50, max_features=100)
        tree.fit(X_bootstrap, y_bootstrap)

        # Append the trained Decision Tree to the list of estimators
        random_forest_model.append(tree)

# Create and train the Random Forest model
train_random_forest(X_train, y_train, random_forest_model)

### Making predictions
A Random Forest predicts by getting individual predictions from all trees, and then it chooses the answer that most trees agree on for classification. This teamwork helps make better predictions and avoids mistakes from just one tree.

Below shows code that is NOT simplified, because later in the exercise you (students) will make a `predict_1_sample` function yourself.

In [270]:
def predict(X, random_forest_model):
    # this code is obfuscated. You will implement this function again in later exercise
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=np.array([tree.predict(X) for tree in random_forest_model]))

# Make predictions on the test set
predictions = predict(X_test, random_forest_model)

### Evaluate accuracy
We can see that the resulting accuracy is now much higher than before, when we just used a single decision tree.

In [271]:
print(y_test==predictions)

N = len(y_test)
count_of_correct_predictions = sum( y_test==predictions  )
accuracy = count_of_correct_predictions/N
print(accuracy)

[ True False False  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True]
0.8809523809523809


### Exercise
1. In our random forest, we have 100 decision trees. Are these 100 trees the same or different from each other? Why?
2. Is it possible that our 100 decision trees all give the same result for a prediction? Why?
3. Is it possible that our 100 decision trees give different results for a prediction? Why?
4. In this case, what should our code do?
5. Make a `predict_1_sample` function, which takes in 1 sonar data `x` and returns the prediction

In [272]:
def predict_1_sample(x, random_forest_model):
  ### code here
  None

In [273]:
# Make predictions on the test set
predictions = []
for x in X_test:
  p = predict(x, random_forest_model)
  predictions.append(p)
print(predictions)

ValueError: ignored

## References
- L. Breiman. Random forests. Maching Learning, 45(1):5–32, Oct. 2001. [[pdf]](https://link.springer.com/content/pdf/10.1023%2FA%3A1010933404324.pdf)
- https://carbonati.github.io/posts/random-forests-from-scratch/
- https://www.tibco.com/reference-center/what-is-a-random-forest
- https://www.youtube.com/watch?v=J4Wdy0Wc_xQ